<a href="https://colab.research.google.com/github/mrninainaidi/Machine-Learning-Projects/blob/master/personal_loan_default_RandForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preamble

* jupyter notebook theme (optional)
* package

In [ ]:
# # Optional: setup theme for Jupyter Notebook
# # comment out if running on Colab
# import jupytertheme as jt
# from jupyterthemes.stylefx import set_nb_theme

# set_nb_theme('chesterish')

In [ ]:
import pandas as pd
import numpy as np
import math

from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import make_scorer
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score

import xgboost
from xgboost import plot_importance

from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK, STATUS_RUNNING

import gc
from scipy import stats
import time
import datetime
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

!pip install missingno
import missingno as msno

!pip install category_encoders
import category_encoders as ce

!pip install imblearn
from imblearn.over_sampling import SMOTE


# import seaborn as sns
# import altair as alt

# Data Processing and Cleaning

## Import Data

In [ ]:
# For colab
from google.colab import drive
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/further_study/machine_learning_projects/personal_loan_rating/'
df = pd.read_csv(root_path+'default_loan_no_quotes.csv')

In [ ]:
# # For others (Jupyter Notebook)
# # NOTE: This requies the data file to be saved under the same directory as this file.

# df = pd.read_csv('default_loan_no_quotes.csv')

In [ ]:
df.head()

In [ ]:
df.columns = df.columns.str.replace(' ','_')
df.columns = map(str.lower, df.columns)
df.columns

## Features With Null Values

In [ ]:
msno.bar(df)

In [ ]:
na_names = list()

for col in df.columns:
    if df[col].isna().sum() > 0:
        print(f'Feature: {col}, has {100 * df[col].isna().sum() / df[col].shape[0]:.3f}%  or {df[col].isna().sum()} null values.')
        na_names.append(col)

na_names

## Checking Feature Types

## Drop the columns of little interests


In [ ]:
'''
Drop the columns of little interests
'''

not_interested = ['entry_date', 'fist_installment_date',\
                  'id', 'deposit_amt', 'financed_amt', 'term_remaining',\
                  'instalment_amt', 'amt_paid_to_merchant_nettofmerchfeesandgst',\
                  'est_fees', 'proc_fees', 'other_fees', 'total_merchant_charges',\
                  'total_consumer_charges']

for name in not_interested: 
    if name not in df.columns:
        raise ValueError(f'column name: {name} is not valid')

df.drop(columns=not_interested, inplace=True)
# df.head()

## Cleaning the consumer post code feature

In [ ]:
# manually correct typos in post code
df['consumer_post_code'].loc[df['consumer_post_code'] == '28501'] = '2850'
df['consumer_post_code'].loc[df['consumer_post_code'] == '2166`1'] = '2166'
df['consumer_post_code'].loc[df['consumer_post_code'] == '414'] = np.nan
df['consumer_post_code'].loc[df['consumer_post_code'] == 'CM144WG'] = np.nan
df['consumer_post_code'].loc[df['consumer_post_code'] == '4Q53'] = '4053'
df['consumer_post_code'].loc[df['consumer_post_code'] == '40/2'] = '4012'
df['consumer_post_code'].loc[df['consumer_post_code'] == '482O'] = '4820'
df['consumer_post_code'].loc[df['consumer_post_code'] == '500O'] = '5000'
df['consumer_post_code'].loc[df['consumer_post_code'] == '430('] = np.nan
df['consumer_post_code'].loc[df['consumer_post_code'] == '48/7'] = '4817'

# convert NA values to 'unknown'
consumerid_list = df['consumer_id'].loc[df['consumer_post_code'].isna()].values
consumerid_list = set(consumerid_list)
print(consumerid_list)

for id in consumerid_list:
    if df['consumer_post_code'].loc[df['consumer_id'] == id].isnull().values.all():
        print(f'consumer: {id} has no post code info')
        df['consumer_post_code'].loc[df['consumer_id'] == id] = 'unknown'
    else:
        possible_post_codes = df["consumer_post_code"].loc[df["consumer_id"] == id].values
        possible_post_codes = possible_post_codes[pd.notna(possible_post_codes)]
        print(f'consumer: {id} has the following post code: {possible_post_codes}')
        print(f'    applying post code to consumer: {id}')
        df['consumer_post_code'].loc[df['consumer_id'] == id] = str(int(possible_post_codes[0]))


# make sure all int and float-type entries are cast to str
df['consumer_post_code'] = df['consumer_post_code'].astype(str).replace('\.0', '', regex=True)
print('Convertion complete.')

In [ ]:
df['consumer_post_code'].value_counts()

In [ ]:
# to check if all instances of the 'consumer_post_code' feature have been converted
# to string-type
for _, row in df.iterrows():
    try: 
        assert(isinstance(row['consumer_post_code'], str))
    except: 
        print(row)

## Cleaning the consumer year of birth feature

In [ ]:
# convert NA values to '99/99/9999'
consumerid_list = df['consumer_id'].loc[df['consumer_year_of_birth'].isna()].values
consumerid_list = set(consumerid_list)

for id in consumerid_list:
    if df['consumer_year_of_birth'].loc[df['consumer_id'] == id].isnull().values.all():
        print(f'consumer: {id} has no DoB info')
        df['consumer_year_of_birth'].loc[df['consumer_id'] == id] = '99/99/9999'
    else:
        
        possibleDoB = df["consumer_year_of_birth"].loc[df["consumer_id"] == id].values
        possibleDoB = possibleDoB[pd.notna(possibleDoB)]
        print(f'consumer: {id} has the following DoBs: {possibleDoB}')
        print(f'    applying DoB to consumer: {id}')
        df['consumer_year_of_birth'].loc[df['consumer_id'] == id] = str(possibleDoB[0])

# Convert str-type DoB to int-type year of birth
df['consumer_year_of_birth'] = df['consumer_year_of_birth'].str.split('/', expand=True)[2].astype(int)
print('Convertion complete.')

In [ ]:
# to check if all instances of the 'consumer_year_of_birth' feature have been converted
# to int-type or np.nan
# for _, row in df.iterrows():
#     yob = row['consumer_year_of_birth']
#     if isinstance(yob, int):
#         if yob > 1900 and yob <= 9999:
#             continue
#     else: 
#         print(f'row["consumer_year_of_birth"] = {yob}')
# print('Assertion complete.')

In [ ]:
x = df['consumer_year_of_birth'].value_counts(normalize=True).sort_index().index
y = df['consumer_year_of_birth'].value_counts(normalize=True).sort_index().values
plt.xlim(1900,2000)
plt.title('Consumer Age Distribution')
plt.xlabel('Year of Birth')
plt.ylabel('Probability')
plt.plot(x, y,'g*')


## Converting application_date feature to application_month and application_year

In [ ]:
# Convert str-type application date to int-type year of application
df['application_year'] = df['application_date'].str.split('/', expand=True)[2].astype(int)
df['application_month'] = df['application_date'].str.split('/', expand=True)[1].astype(int)


## Converting recent_default_default_date to recent_default_year and recent_dafault_month

In [ ]:
df['recent_default_default_date'] = df['recent_default_default_date'].replace(np.nan, '00/00/0000', regex=True)
df['recent_default_year'] = df['recent_default_default_date'].str.split('/', expand=True)[2].astype(int)
df['recent_default_month'] = df['recent_default_default_date'].str.split('/', expand=True)[1].astype(int)
# df['recent_default_month'].value_counts()

In [ ]:
x = df['recent_default_month'].value_counts(normalize=True).sort_index().index
y = df['recent_default_month'].value_counts(normalize=True).sort_index().values

plt.xlim(1,12)
plt.ylim(0,0.02)

plt.title('Recent default month distribution')
plt.xlabel('Month of default')
plt.ylabel('Probability')
plt.plot(x, y,'g*-')

## Adding age_of_application feature (integer)

In [ ]:
df['age_of_application'] = df['application_year'] - df['consumer_year_of_birth']

# use this "age of application" to validate the "consumer year of birth"
# if "age of application" < 18, the minimum legal age of having a credit account
# the "consumer year of birth" entry must be faulty. 
df['consumer_year_of_birth'].loc[df['age_of_application'] < 18] = int(9999)
df['age_of_application'].loc[df['age_of_application'] < 18] = int(-1)

In [ ]:
x = df['age_of_application'].value_counts(normalize=True).sort_index().index
y = df['age_of_application'].value_counts(normalize=True).sort_index().values
plt.xlim(-1,100)
plt.title('Consumer Age of Application')
plt.xlabel('Consumer Age')
plt.ylabel('Probability')
plt.plot(x, y, 'g*')

## Adding age_of_recent_default feature (integer)

In [ ]:
df['age_of_recent_default'] = df['recent_default_year'] - df['consumer_year_of_birth']
# df['age_of_recent_default'].value_counts(normalize=True)

In [ ]:
# recent default should not happen before the year of application
# the age of application has to be > 18 for age of recent default to be effective

# For invalid entry of recent_default_default_date and consumer_year_of_birth
df['age_of_recent_default'].loc[(df['age_of_recent_default'] <df['age_of_application'])\
                                | (df['age_of_application'] < 18)] = int(-1)
df['recent_default_year'].loc[(df['age_of_recent_default'] <df['age_of_application'])\
                                | (df['age_of_application'] < 18)] = int(0)
df['recent_default_month'].loc[(df['age_of_recent_default'] <df['age_of_application'])\
                                | (df['age_of_application'] < 18)] = int(0)

# For absent recent_default_default_date 
df['age_of_recent_default'].loc[df['recent_default_year'] == 0] = int(0)
df['recent_default_year'].loc[df['recent_default_year'] == 0] = int(0)
df['recent_default_month'].loc[df['recent_default_year'] == 0] = int(0)

print('Convertion complete.')

In [ ]:
x = df['age_of_recent_default'].value_counts(normalize=True).sort_index().index
y = df['age_of_recent_default'].value_counts(normalize=True).sort_index().values
plt.xlim(20,100)
plt.ylim(0,0.01)
plt.title('Consumer Age of Recent Default')
plt.xlabel('Consumer Age')
plt.ylabel('Probability')
plt.plot(x, y, 'g*')

## Cleaning product feature

In [ ]:
# replace NaN with 'unknown'
df['product'] = df['product'].replace(np.nan, 'unknown', regex=True)

In [ ]:
# shorten the tails
x = df['product'].value_counts(normalize=True).index
y = df['product'].value_counts(normalize=True).values



## Shorten the features with heavy tails

* 'product'
* 'merchant_name'
* 'merchant_number'

In [ ]:

def convert_tails_to_others(dataframe, feature, fracToConvert):
    x = dataframe[feature].value_counts(normalize=True).index
    y = dataframe[feature].value_counts(normalize=True).values

    all_list = dataframe[feature].value_counts(normalize=True).index.tolist()

    # obtain the list of value to keep
    threshold = 1 - fracToConvert
    current = 0.0
    keep_list = list()

    for i in range(len(y)):
        if current >= threshold:
            break
        current += y[i]
        keep_list.append(x[i])

    drop_list = [x for x in all_list if x not in keep_list]

    # apply keep_list
    dataframe[feature].loc[dataframe[feature].isin(drop_list)] = 'others'
    # print(dataframe[feature].value_counts(normalize=True))
    # print()

In [ ]:
col_names = ['product', 'merchant_name', 'merchant_number']
frac_dict = {'product':0.08, 'merchant_name':0.05, 'merchant_number':0.05}

for name in col_names:
    convert_tails_to_others(df, name, frac_dict[name])

## Cleaning total_balance_outstanding feature

In [ ]:
df_tmp = df['total_balance_outstanding']

df_tmp.replace(np.nan, '0.0', regex=True, inplace=True)
df_tmp.replace(',', '', regex=True, inplace=True)
df_tmp = df_tmp.astype(float)

df['total_balance_outstanding'] = df_tmp
del df_tmp
print('Convertion complete')

In [ ]:
# # check if everything has been converted to float-type
# for index, value in df['total_balance_outstanding'].items():
#     if not isinstance(value, float):
#         print(f'{value} ----- {type(value)}')
# print('Assertion complete.')

## Cleaning recent_default_default_amt feature

In [ ]:
df_tmp = df['recent_default_default_amt']

df_tmp.replace(np.nan, '0.0', regex=True, inplace=True)
df_tmp.replace(',', '', regex=True, inplace=True)
df_tmp = df_tmp.astype(float)

df['recent_default_default_amt'] = df_tmp
del df_tmp
print('Convertion complete')

In [ ]:
# # check if everything has been converted to float-type
# for index, value in df['recent_default_default_amt'].items():
#     if not isinstance(value, float):
#         print(f'{value} ----- {type(value)}')
# print('Assertion complete.')

## Adding term_run_frac feature
representing the fraction of terms that have been fulfilled. 

In [ ]:
df['term_run_frac'] = df['term_run'] / df['total_term']

## Adding total_month feature

In [ ]:
df_tmp = pd.DataFrame()
df_tmp['total_term'] = df['total_term']
df_tmp['total_month'] = df['total_term']
df_tmp['freq'] = df['freq']

mask = (df_tmp['freq'] == 'FN')
df_valid = df_tmp[mask]

df_tmp.loc[mask, 'total_month'] = df_valid['total_term'] / 2

df['total_month'] = df_tmp['total_month']
del df_tmp

## Exploring other features

### Exploring arrears amount feature

In [ ]:
# a = df['arrears_amount'].value_counts(dropna=False).sort_index()
# print(a)

In [ ]:
# df_tmp = df.loc[ df['arrears_amount'] == 0]
# df_tmp['contract_status'].value_counts(normalize=True, dropna=False)

### Exploring contract_status feature

In [ ]:
# '''
# Separate the dataframe by contract status
# '''

# df_paid = df.loc[df['contract_status'] == 'PaidInFull']
# df_default = df.loc[df['contract_status'] == 'Default']
# df_active = df.loc[df['contract_status'] == 'Active']
# print(df['contract_status'].value_counts(normalize=True))

In [ ]:
# # '''
# # Look at the Paid-set
# # '''
# print(df_paid['arrears_amount'].value_counts(dropna=False, normalize=True).sort_index())
# print()
# print(df_paid['age_of_recent_default'].value_counts(dropna=False, normalize=False).sort_index())
# print()
# print(df_paid['total_balance_outstanding'].value_counts(dropna=False, normalize=True).sort_index())
# print()
# print(df_paid['defaultdate'].value_counts(dropna=False, normalize=True).sort_index())
# print()
# print((df_paid['term_run'] / df_paid['total_term']).value_counts(dropna=False, normalize=True).sort_index())
# print()


In [ ]:
# # '''
# # Look at the Default-set
# # '''
# print(df_default['arrears_amount'].value_counts(dropna=False, normalize=True).sort_values(ascending=False))
# print()
# print(df_default['age_of_recent_default'].value_counts(dropna=False, normalize=True).sort_values(ascending=False))
# print()
# print(df_default['total_balance_outstanding'].value_counts(dropna=False, normalize=True).sort_values(ascending=False))
# print()
# print(df_default['defaultdate'].value_counts(dropna=False, normalize=True).sort_index())
# print()
# print((df_default['term_run'] / df_default['total_term']).value_counts(dropna=False, normalize=True).sort_index())
# print()

In [ ]:
# # '''
# # Look at the active-set
# # '''
# print(df_active['arrears_amount'].value_counts(dropna=False, normalize=True).sort_index())
# print()
# print(df_active['age_of_recent_default'].value_counts(dropna=False, normalize=True).sort_index())
# print()
# print(df_active['total_balance_outstanding'].value_counts(dropna=False, normalize=True).sort_index())
# print()
# print(df_active['defaultdate'].value_counts(dropna=False, normalize=True).sort_index())
# print()
# print((df_active['term_run'] / df_active['total_term']).value_counts(dropna=False, normalize=True).sort_index())
# print()

**My Questions regarding the "contract_status" feature (ground-truth?)**

* All instances in "PaidInFull" subset have 0.0 "arrears_amount", but more than 10% of the instances in this subset has a valid "age_of_recent_default", which suggests default did occur to these intances. All instances in this subset have NaN value for "defaultdate"

* About 3% of the instances in "Default" subset have 0.0 "arrears_amount", which indicate default has never occured to these instances. 99.7% of this subset have a valid "defaultdate" entry.

* "Active" subset has very similar behaviour when compared against "PaidInFull". All instances in this subset have NaN value for "defaultdate"

**Options for ground-truth**

If **True** ==> has default, **False** ==> has no default: 

1. **True** = "Default" subset and **False** = "PaidInFull" + "Active" subsets

2. **True** = "arrears_amount" != 0, and **False** = "arrears_amount" == 0

3. **True** = "defaultdate" == valid date, and **False** = NaN

**According to the observations, none of the above options are fully make sense...**

**Meeting outcome**


4. use the combined recent group to determine the ground truth. 

    i.e. when "age_of_recent_default", "age_of_recent_default_cure" and "recent_default_amt" are all valid ==> **DEFAULT**

## Define ground truth

In [ ]:
# df_recent = df[[col for col in df.columns if 'recent' in col]]
# df_recent['defaultdate'] = df['defaultdate']
# df_recent['consumer_id'] = df['consumer_id']
# df_recent['defaultamount'] = df['defaultamount']
# df_recent['contract_number'] = df['contract_number']
# df_recent['contract_status'] = df['contract_status']
# df_recent['expected_contract_end_date'] = df['expected_contract_end_date']

In [ ]:
# for index, row in df_recent.iterrows():
#     if row['recent_default_default_amt'] == 0:
#         if isinstance(row['defaultdate'], str):
#             print(row)
#             print()      

**Test outcome**

* when "recent_default_year" == 0, there are FIVE instances that "recent_default_default_amt" != 0. And all FIVE instances are marked as DEFAULT by the "contract_status"


* when "recent_default_default_amt" == 0, there are TWO instances that "recent_default_year" != 0. And all of the TWO instances are marked as PAIDINFULL by the "contract_status".

    ==> both "recent_default_year" and "recent_default_default_amt" == 0 means NoDefault

**Suspect bad columns:**

Assumning 'recent_default_default_amt' is the indicator for the ground truth... 

* 'defaultdate'

* 'defaultamount'

* 'total_balance_outstanding'

* 'recent_default_default_date' ==> 'recent_default_year' ==> 'recent_default_age'


**Question:**

Do I go back to realign 'recent_default_year' and 'recent_default_age' with the assumed ground truth???


In [ ]:
# introduce the ground truth according to above analysis
df['isDefault'] = df['recent_default_default_amt'] > 0

In [ ]:
# # to check
# for index, row in df.iterrows():
#     if row['recent_default_default_amt'] > 0:
#         if df['isDefault'] is False:
#             print(row)
# print('Assertion complete.')

## Finalising data processing and digitising categorical features

In [ ]:
print(df.columns)

### Collect numeric features

In [ ]:
df_train = pd.DataFrame()

num_names = ['purchase_amt','deposit_percent', 'age_of_application', \
             'gtee_rate','term_run_frac', 'total_month']

for name in num_names: 
    if name not in df.columns:
        raise ValueError(f'column name: {name} is not valid')

for name in num_names: 
    df_train[name] = df[name].copy()

df_train.head(10)

### Encode categorical features

In [ ]:
def ordered_labels(df, col, order):
    df[col] = df[col].astype('category')
    df[col] = df[col].cat.reorder_categories(order, ordered=True)
    df[col] = df[col].cat.codes.astype(int)

In [ ]:
# split the categorical features into... 
# ordered
ordered_features = ['application_year', 'application_month']
ordered_dict = dict()
for elm in ordered_features:
    ordered_dict[elm] = df[elm].value_counts().sort_index().index

# one-hot like
oneHot_features = ['product', 'consumer_post_code', 'industry_name']

# others
nominated_features = ['payment_type', 'freq', 'homowner_ind', 'homowner_consumer', 'isDefault']

cate_names = ordered_features + nominated_features + oneHot_features

# initialise encoder
le = LabelEncoder()

# start encoding... 
for col in cate_names: 
    df_train[col] = df[col].copy()

    # the ordered_features
    if col in ordered_features:
        ordered_labels(df_train, col, ordered_dict[col])
        continue

    # the one-hot like features
    if col in oneHot_features:
        encoder = ce.BinaryEncoder(cols=[col])
        df_train = pd.concat([df_train, encoder.fit_transform(df_train[col])], axis=1)
        df_train.drop(columns=[col], inplace=True)
        continue

    # other features
    le.fit(list(df_train[col].astype(str).values))
    df_train[col] = le.transform(list(df_train[col].astype(str).values))
    length = df_train[col].value_counts().shape

df_train.head(10)

In [ ]:
df_train.shape

### Rescale everything to (0, 1)

In [ ]:
# Scale numeric features to (0, 1)
value_array = df_train.values
col_names = df_train.columns

min_max_scaler = MinMaxScaler()

value_array_scaled = min_max_scaler.fit_transform(value_array)

df_train = pd.DataFrame(value_array_scaled, columns=col_names)

df_train.head(10)

### Separate input and output

In [ ]:
# model input
X = df_train.drop(columns=['isDefault'])

# expected output
y = df_train['isDefault']

del df_train
print(X.shape)
print(y.shape)

# Model Training and Validating

## HyperOpt function and parameters space definition

In [ ]:
'''
Define the objective function
    The output of this objective fnc has been set to the negative value of the mean
    score, so that fmin() can be used to find the most suitable parameters for 
    maximum mean score.
'''
def objective(params):
    time1 = time.time()

    params = {
        'n_estimators'    : int(params['n_estimators']),
        'max_features'    : int(params['max_features'])
    }

    df_toprint = pd.DataFrame(params, index=[0])

    print('\n############## New Run ################')
    print(f"params = {df_toprint.transpose()}")

    # declair total number of folds and fold counter
    FOLDS = 6
    counter = 1

    # instantiate the TSS model
    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)

    # initialise the 0-valued array to hold the flag of wrong predictions for training set
    y_oof = np.zeros(X.shape[0])

    # initialise the mean score for the cross-validation
    score_mean = 0

    # initialise SMOTE sampling
    sm = SMOTE(random_state=3)

    print('\nCV - scores: ')

    # Start the Training and Cross-validation loop
    for t_idx, v_idx in skf.split(X, y):

        # instantiate the XGB classifier
        clf = RandomForestClassifier(random_state=3, **params)

        # get the time series split indices
        X_t, X_v = X.iloc[t_idx, :], X.iloc[v_idx, :]
        y_t, y_v = y.iloc[t_idx], y.iloc[v_idx]

        # # resample the training set
        # X_t_res, y_t_res = sm.fit_sample(X_t, y_t.ravel())

        # Model training
        clf.fit(X_t, y_t)
        # clf.fit(X_t_res, y_t_res)

        # Obtain the validation score for the fitted model
        score = make_scorer(roc_auc_score, needs_proba=True)(clf, X_v, y_v)
        score_mean += score

        print(f'    {counter} : {round(score, 4)};', end = " "),


        # advance the fold counter
        counter += 1
    
    # record the time elapsed
    time2 = time.time() - time1

    print(f"Total Time Run: {round(time2 / 60,2)} mins")
    gc.collect() # garbage collection
    print(f'Mean ROC_AUC: {score_mean / FOLDS}')

    del X_t, X_v, y_t, y_v, clf, score
    # del X_t_res, y_t_res


    return -(score_mean / FOLDS)

In [ ]:
'''
Initial guess of the objective function parameters
'''
space = {
    'n_estimators' : hp.choice('n_estimators', list(range(10,200,1))), 
    'max_features' : hp.choice('max_features', list(range(3,46,1)))
}

## Running the optimiser to train the model and obtain the best parameter combination

In [ ]:
# Set algoritm parameters
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=40)

# Print best parameters
best_params = space_eval(space, best)

print("BEST PARAMS: ", best_params)

# Post Processing -- Feature Importance Summary

## Train a model with the best parameter set

In [ ]:
clf = RandomForestClassifier(random_state=3, **best_params)

clf.fit(X, y)

## Obtain the feature importance map

In [ ]:
# get feature importance ratings
importance = list(map(abs, clf.coef_[0].tolist()))
feat_names = X.columns

df_featImp = pd.DataFrame(data=importance, index=feat_names, columns=['score']).sort_values(by='score', ascending=True)


ax = df_featImp.plot(kind='barh', figsize=(5,10))

In [ ]:
df_featImp.sort_values(by='score', ascending=False).head(20)

In [ ]:
# terminates the current run
raise SystemExit('Run Terminated.') 

# Training Results

In [ ]:
'''
BEST PARAMS:  {'C': 545.5594781168514, 'penalty': 'l2', 'solver': 'liblinear'}


CV - scores: 
    1 : 0.6513;
    2 : 0.6523;
    3 : 0.6509;
    4 : 0.6522;
    5 : 0.6554;
    6 : 0.65;
Total Time Run: 0.38


age_of_application	    3.305887
purchase_amt	        3.111321
deposit_percent	        1.750232
gtee_rate	            0.644461
homowner_consumer	    0.505763
total_month	            0.448497
industry_name_1	        0.404635
industry_name_4	        0.357544
term_run_frac	        0.305157
application_year	    0.282223
consumer_post_code_2	0.248970
payment_type	        0.246252
consumer_post_code_1	0.220856
product_2	            0.212898
product_1	            0.182107
product_3	            0.124898
industry_name_2	        0.106117
homowner_ind	        0.104002
industry_name_6 	    0.092173
consumer_post_code_3	0.079214

'''

In [ ]:
'''
BEST PARAMS:  {'C': 16.23776739188721, 'penalty': 'l2', 'solver': 'lbfgs'}


CV - scores: 
    1 : 0.6513;
    2 : 0.6523;
    3 : 0.6509;
    4 : 0.6522;
    5 : 0.6553;
    6 : 0.6501;


age_of_application	    3.311348
purchase_amt	        3.095686
deposit_percent	        1.746661
gtee_rate	            0.651738
industry_name_1	        0.527772
homowner_consumer	    0.491427
total_month	            0.452777
industry_name_4	        0.362157
application_year	    0.290661
consumer_post_code_2	0.249201
payment_type	        0.246723
consumer_post_code_1	0.221004
product_2	            0.215359
term_run_frac	        0.215209
product_1	            0.174818
product_3	            0.126728
industry_name_2	        0.102357
industry_name_6	        0.094139
homowner_ind	        0.084542
consumer_post_code_3	0.079851
'''

# Code Dump